# End-to-End Pass Receiver Prediction on Video

This notebook contains code to perform end-to-end inference on a sample clip. We load the clip and perform each stage sequentially. Note that this is not intended to be real-time.

In [1]:
# Imports
import os
import sys

sys.path.append('./PerspectiveTransform')

from ultralytics import YOLO
import pandas as pd
from tqdm import tqdm
import numpy as np
from clustimage import Clustimage
from matplotlib import image
import matplotlib.pyplot as plt
from collections import Counter
import cv2

from elements.perspective_transform import Perspective_Transform
from elements.assets import transform_matrix, detect_color
from elements.args import Args

import mat
import torch

## Player and Ball Detection

In [2]:
# Build and load model (Yolov8l without pretraining, 1080 resolution)
Yolov8Detector = YOLO('runs/YOLOv8/yolov8l_soccerpass_fixed_res_1080_nopretrain/weights/best.pt')

device = 0 #0,1,2=cuda device 0,1,2 else 'cpu'

In [3]:
results = Yolov8Detector.predict(source='downloads/sample1.mp4', device=device)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 99.0ms
video 1/1 (2/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 82.3ms
video 1/1 (3/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 86.8ms
video 1/1 (4/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football

video 1/1 (66/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 62.2ms
video 1/1 (67/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 60.7ms
video 1/1 (68/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 2 Footballs, 60.2ms
video 1/1 (69/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 2 Footballs, 60.8ms
video 1/1 (70/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 2 Footballs, 60.2ms
video 1/1 (71/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 2 Footballs, 61.0ms
video 1/1 (72/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 2 Footballs, 59.9ms
video 1/1 (73/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 60.1ms
video 1/1 (74/725) /home/sn3007/FootballPassPredict

video 1/1 (137/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 61.4ms
video 1/1 (138/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 23 Players, 61.4ms
video 1/1 (139/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 60.7ms
video 1/1 (140/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 1 Football, 62.7ms
video 1/1 (141/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 24 Players, 1 Football, 59.5ms
video 1/1 (142/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 23 Players, 1 Football, 61.9ms
video 1/1 (143/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 24 Players, 1 Football, 60.7ms
video 1/1 (144/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 25 Players, 1 Football, 62.0ms
video 1/1 (145/725) /home/sn3007/FootballPassPrediction/downloads/sample

video 1/1 (212/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 23 Players, 61.2ms
video 1/1 (213/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 23 Players, 1 Football, 59.5ms
video 1/1 (214/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 23 Players, 62.9ms
video 1/1 (215/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 23 Players, 60.4ms
video 1/1 (216/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 63.5ms
video 1/1 (217/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 60.9ms
video 1/1 (218/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 61.7ms
video 1/1 (219/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 61.2ms
video 1/1 (220/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 60.2ms
video 1/1 (221/725) /home

video 1/1 (283/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 20 Players, 1 Football, 60.7ms
video 1/1 (284/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 61.1ms
video 1/1 (285/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 61.3ms
video 1/1 (286/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 60.4ms
video 1/1 (287/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 63.8ms
video 1/1 (288/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 61.1ms
video 1/1 (289/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 62.0ms
video 1/1 (290/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 60.9ms
video 1/1 (291/725) /home/sn3007/FootballPassPre

...

video 1/1 (425/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 62.5ms
video 1/1 (426/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 59.9ms
video 1/1 (427/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 62.9ms
video 1/1 (428/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 1 Football, 60.8ms
video 1/1 (429/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 21 Players, 62.3ms
video 1/1 (430/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 1 Football, 61.7ms
video 1/1 (431/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 1 Football, 61.7ms
video 1/1 (432/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 22 Players, 1 Football, 64.6ms
video 1/1 (433/725) /home/sn3007/FootballPassPrediction/downloads/sample

video 1/1 (495/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 61.4ms
video 1/1 (496/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 63.1ms
video 1/1 (497/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 60.7ms
video 1/1 (498/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 63.2ms
video 1/1 (499/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 61.7ms
video 1/1 (500/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 61.2ms
video 1/1 (501/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 62.5ms
video 1/1 (502/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 60.8ms
video 1/1 (503/725) /home/sn3007/FootballPassPre

video 1/1 (565/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 61.4ms
video 1/1 (566/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 62.3ms
video 1/1 (567/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 60.7ms
video 1/1 (568/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 1 Football, 62.0ms
video 1/1 (569/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Players, 63.5ms
video 1/1 (570/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 61.8ms
video 1/1 (571/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 61.6ms
video 1/1 (572/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 63.3ms
video 1/1 (573/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 15 Playe

video 1/1 (637/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 20 Players, 1 Football, 61.5ms
video 1/1 (638/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 61.3ms
video 1/1 (639/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 64.0ms
video 1/1 (640/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 61.0ms
video 1/1 (641/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 63.2ms
video 1/1 (642/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 2 Footballs, 63.9ms
video 1/1 (643/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 60.7ms
video 1/1 (644/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 19 Players, 1 Football, 63.4ms
video 1/1 (645/725) /home/sn3007/FootballPassPr

video 1/1 (708/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 62.1ms
video 1/1 (709/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 62.1ms
video 1/1 (710/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 64.0ms
video 1/1 (711/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 61.7ms
video 1/1 (712/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 63.0ms
video 1/1 (713/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 63.3ms
video 1/1 (714/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 61.3ms
video 1/1 (715/725) /home/sn3007/FootballPassPrediction/downloads/sample1.mp4: 640x1088 14 Players, 1 Football, 64.4ms
video 1/1 (716/725) /home/sn3007/FootballPassPre

In [4]:
print(len(results))

725


`len(results)` represents the number of frames read from the input clip. We generate an annotation dataframe containing the predictions for each frame.

In [5]:
detect_df = [['pred_class', 'pred_x', 'pred_y', 'pred_w', 'pred_h', 'image_name', 'OID']] #Headers

for frame,res in enumerate(results): #Iterate results of prediction
    max_ball_conf = 0.
    max_ball_conf_row = None
    for i, box in enumerate(res.boxes):
        oid = i+1
        coords = box.xywhn.detach().cpu()
        cls = box.cls.detach().cpu()
        cls = int(cls.data[0])
        conf = box.conf.detach().cpu()
        conf = float(conf.data[0])
        if cls == 1 and conf > max_ball_conf: #Get max conf ball out of all ball predictions for image
            max_ball_conf = conf
            max_ball_conf_row = [cls ,coords[0][0].item(), coords[0][1].item(), coords[0][2].item(), coords[0][3].item(), frame, oid]
        elif cls == 0: #For player, just directly add the prediction
            row = [cls ,coords[0][0].item(), coords[0][1].item(), coords[0][2].item(), coords[0][3].item(), frame, oid]
            detect_df.append(row)
    if max_ball_conf_row is not None:
        detect_df.append(max_ball_conf_row)
        
headers = detect_df.pop(0)
detect_df = pd.DataFrame(detect_df, columns=headers) #Convert to df
detect_df

,pred_class,pred_x,pred_y,pred_w,pred_h,image_name,OID
0,0,0.902366,0.795507,0.039593,0.097100,0,1
1,0,0.640834,0.530382,0.016768,0.078459,0,2
2,0,0.858254,0.601951,0.019514,0.098764,0,3
3,0,0.237743,0.736137,0.020812,0.095080,0,4
4,0,0.390634,0.391285,0.017607,0.066016,0,5
...,...,...,...,...,...,...,...
14413,0,0.783640,0.308762,0.042718,0.117798,724,11
14414,0,0.287292,0.367956,0.025289,0.114402,724,12
14415,0,0.299241,0.446686,0.044959,0.116431,724,13
14416,0,0.401181,0.190410,0.028988,0.099469,724,14


In [6]:
# Frames are represented by "image_name" column. We drop frames where the ball is not detected.
groupdf=detect_df.groupby("image_name")
ndetect=[]
dropped=[]
for name,df in groupdf:
    if len(df[df["pred_class"]==1])==1:
        ndetect.append(df)
    else:
        dropped.append(name)
ndetect_df=pd.concat(ndetect)
ndetect_df=ndetect_df.dropna()

## Team Identification

In [9]:
def pre_process_crop(image, bbox):
    H = image.shape[0]
    W = image.shape[1]

    x = int(bbox[0]*W)
    y = int(bbox[1]*H)
    w = int(bbox[2]*W)
    h = int(bbox[3]*H)
    
    #Extracting Crop from given image with bbox specs
    crop = image[y:y+h, x:x+w]
    crop = cv2.resize(crop, (20,40))

    #Applying filters to the crop
    crop = cv2.medianBlur(crop,3)
    crop = cv2.bilateralFilter(crop,5,30,30)
    
    return crop

In [10]:
def formatPred(pred):
    y = np.unique(pred, return_counts=True)
    z = sorted(zip(y[1], y[0]), reverse=True)
    res = pred.copy()
    i=0
    for tup in z:
        idx_list = np.where(pred == tup[1])[0]

    # print(tup, idx_list)
        for idx in idx_list:
            res[idx] = i
        i+=1

    return res

In [11]:
# Clustering function for team identification
def cluster(yolo_df):
    total_boxes = 0
    total_misclassified = 0

    groups = yolo_df.groupby("image_name")
    
    video_path = 'downloads/sample1.mp4'
    cap = cv2.VideoCapture(video_path)

  # loop through each group and apply the code snippet to each image
    for image_name, group in tqdm(groups):
        frame_id=int(image_name)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        img = frame
        players_info = group[group['pred_class'] == 0]

        min_num_cats = 3
        max_num_cats = 4

        player_boxes = []

        for index, row in players_info.iterrows():

            bbox = [row['pred_x']-(row['pred_w']/2), row['pred_y']-(row['pred_h']/2), row['pred_w'], row['pred_h']]
            crop = pre_process_crop(img, bbox)
            player_boxes.append(crop)

        player_boxes_reshaped = np.array(player_boxes).reshape(len(player_boxes),-1)
        player_boxes_reshaped = player_boxes_reshaped/255.0

        cl = Clustimage(method='pca',
                    embedding='tsne',
                    grayscale=False,
                    dim=(20,40),
                    params_pca={'n_components':0.95},
                    verbose=60)

        results = cl.fit_transform(player_boxes_reshaped,
                                cluster='agglomerative',
                                evaluate='silhouette',
                                metric='euclidean',
                                linkage='ward',
                                min_clust=min_num_cats,
                                max_clust=max_num_cats,
                                cluster_space='high')

        pred_labels = results['labels']

        preds = formatPred(pred_labels)

        i = 0
        for index, row in players_info.iterrows():
            if(int(row['pred_class']) != 0):
                continue
            yolo_df.loc[index, 'pred_team'] = int(preds[i])
            i+=1
        print("name: {}, preds: {}, total boxes in : {}".format(image_name, preds, players_info.shape[0]))
        total_boxes += players_info.shape[0]

    print("total_boxes: {} ,  total_images: {}".format(total_boxes, len(groups)))

    return yolo_df

In [12]:
clustered_df = cluster(ndetect_df)

  0%|                                                                                           | 0/537 [00:00<?, ?it/s]

(14, 2400)
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [10] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.

[clusteval] >INFO: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 596.80it/s]
[clustimage] >DEBUG> [[1 3 1 1 1 2 1 3 2 1 3 3 1 1]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [    0.35687]
[clustimage] >DEBUG> sillclust: [          3]
[clustimage] >DEBUG> clustlabx: [[1 3 1 1 1 2 1 3 2 1 3 3 1 1]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.
  0%|▏                                                                                  | 1/537 [00:00<05:42,  1.56it/s]

name: 0, preds: [0 1 0 0 0 2 0 1 2 0 1 1 0 0], total boxes in : 14
(14, 2400)
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [10] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.

[clusteval] >INFO: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 550.29it/s]
[clustimage] >DEBUG> [[1 2 1 2 1 1 1 3 2 2 1 1 2 1]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [    0.30694]
[clustimage] >DEBUG> sillclust: [          3]
[clustimage] >DEBUG> clustlabx: [[1 2 1 2 1 1 1 3 2 2 1 1 2 1]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.
  0%|▎                                                                                  | 2/537 [00:00<03:46,  2.36it/s]

...

name: 722, preds: [1 1 0 1 1 0 0 0 0 0 1 1 0 2], total boxes in : 14
(14, 2400)
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [10] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.

[clusteval] >INFO: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 591.66it/s]
[clustimage] >DEBUG> [[1 2 2 2 2 1 1 1 1 2 2 1 1 3]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [     0.3282]
[clustimage] >DEBUG> sillclust: [          3]
[clustimage] >DEBUG> clustlabx: [[1 2 2 2 2 1 1 1 1 2 2 1 1 3]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.
100%|████████████████████████████████████████████████████████████████████████████████▊| 536/537 [04:13<00:00,  2.07it/s]

name: 723, preds: [0 1 1 1 1 0 0 0 0 1 1 0 0 2], total boxes in : 14
(14, 2400)
[pca] >Column labels are auto-completed.
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [2400] columns of the input data.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.
[pca] >Compute explained variance.
[pca] >Number of components is [10] that covers the [95.00%] explained variance.
[pca] >The PCA reduction is performed on the [2400] columns of the input dataframe.
[pca] >Fit using PCA.
[pca] >Compute loadings and PCs.


[clustimage] >INFO> Saving data in memory.
[clustimage] >INFO> Fit with method=[agglomerative], metric=[euclidean], linkage=[ward]
[clustimage] >INFO> Evaluate using silhouette.

[clusteval] >INFO: 100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 553.48it/s]
[clustimage] >DEBUG> [[2 2 2 2 2 1 1 1 1 1 2 1 1 3]]
[clustimage] >DEBUG> Iloc: [ True]
[clustimage] >DEBUG> silscores: [     0.3458]
[clustimage] >DEBUG> sillclust: [          3]
[clustimage] >DEBUG> clustlabx: [[2 2 2 2 2 1 1 1 1 1 2 1 1 3]]
[clustimage] >INFO> Compute dendrogram threshold.
[clustimage] >INFO> Optimal number clusters detected: [3].
[clustimage] >INFO> Fin.
100%|█████████████████████████████████████████████████████████████████████████████████| 537/537 [04:13<00:00,  2.12it/s]

name: 724, preds: [1 1 1 1 1 0 0 0 0 0 1 0 0 2], total boxes in : 14
total_boxes: 10169 ,  total_images: 537


In [13]:
clustered_df

,pred_class,pred_x,pred_y,pred_w,pred_h,image_name,OID,pred_team
0,0,0.902366,0.795507,0.039593,0.097100,0,1,0.0
1,0,0.640834,0.530382,0.016768,0.078459,0,2,1.0
2,0,0.858254,0.601951,0.019514,0.098764,0,3,0.0
3,0,0.237743,0.736137,0.020812,0.095080,0,4,0.0
4,0,0.390634,0.391285,0.017607,0.066016,0,5,0.0
...,...,...,...,...,...,...,...,...
14413,0,0.783640,0.308762,0.042718,0.117798,724,11,1.0
14414,0,0.287292,0.367956,0.025289,0.114402,724,12,0.0
14415,0,0.299241,0.446686,0.044959,0.116431,724,13,0.0
14416,0,0.401181,0.190410,0.028988,0.099469,724,14,2.0


## Perspective Transformation

We load and use the perspective transform model to obtain (x, y) coordinates for each player and the ball; we use Euclidean Distance to determine the player with ball possession.

In [14]:
#Load Arguments for perspective transform model
opt = Args()

# Load model
pt_model = Perspective_Transform(opt)

Perspective Transform model loaded!


In [15]:
def generate_perspective_transform(model, image_path, yolo_df):

    groups = yolo_df.groupby("image_name")
    video_path = 'downloads/sample1.mp4'
    capt = cv2.VideoCapture(video_path)
    # loop through each group and apply the code snippet to each image
    for image_name, group in tqdm(groups):
        frame_id=int(image_name)
        capt.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = capt.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cap = frame

        w = cap.shape[1] #1280
        h = cap.shape[0] #720

        main_frame = cap.copy()

        M, warped_image = model.homography_matrix(main_frame)


        for index, row in group.iterrows():
            if(math.isnan(row['pred_class'])):
                continue
            else:
                x_center = row['pred_x'] #if x is center
                y_center = row['pred_y'] + (row['pred_h']/2)

                coords = transform_matrix(M, (x_center*w, y_center*h), (h, w), (68, 105))

                yolo_df.loc[index, 'gnd_x'] = coords[0]
                yolo_df.loc[index, 'gnd_y'] = coords[1]
        
    return yolo_df


In [16]:
with torch.no_grad():
    transformed_df = generate_perspective_transform(pt_model, None, clustered_df)
# For image and all bbox+ID => return ID +x+y coordinates

 45%|████████████████████████████████████▎                                            | 241/537 [05:45<11:22,  2.31s/it]

100%|█████████████████████████████████████████████████████████████████████████████████| 537/537 [13:28<00:00,  1.50s/it]


In [17]:
transformed_df

,pred_class,pred_x,pred_y,pred_w,pred_h,image_name,OID,pred_team,gnd_x,gnd_y
0,0,0.902366,0.795507,0.039593,0.097100,0,1,0.0,77.0,55.0
1,0,0.640834,0.530382,0.016768,0.078459,0,2,1.0,73.0,38.0
2,0,0.858254,0.601951,0.019514,0.098764,0,3,0.0,79.0,45.0
3,0,0.237743,0.736137,0.020812,0.095080,0,4,0.0,57.0,50.0
4,0,0.390634,0.391285,0.017607,0.066016,0,5,0.0,65.0,24.0
...,...,...,...,...,...,...,...,...,...,...
14413,0,0.783640,0.308762,0.042718,0.117798,724,11,1.0,34.0,0.0
14414,0,0.287292,0.367956,0.025289,0.114402,724,12,0.0,23.0,9.0
14415,0,0.299241,0.446686,0.044959,0.116431,724,13,0.0,25.0,14.0
14416,0,0.401181,0.190410,0.028988,0.099469,724,14,2.0,23.0,-5.0


In [18]:
#Ball Possession Identification using distance. 
#The player closest to the ball will be marked as the current dribbler.
def Ball_Possession(transformed_df):
    def calc_distance(row, ballx, bally):
        x2 = row['gnd_x']
        y2 = row['gnd_y']
        distance = np.sqrt((ballx - x2)**2 + (bally - y2)**2)
        return distance

    transformed_df['gnd_x'] = transformed_df['gnd_x'].apply(lambda x: max(x, 0))
    transformed_df['gnd_y'] = transformed_df['gnd_y'].apply(lambda x: max(x, 0))
    
    transformed_df['bp']=0
    transformed_df['distance']=0

    grouped_df=transformed_df.groupby('image_name')


    for name,group in grouped_df:

        row = group.loc[group['pred_class'] == 1, ['gnd_x', 'gnd_y']].iloc[0]
        ballx=row['gnd_x']
        bally=row['gnd_y']
        group['distance'] = group.apply(calc_distance, args=(ballx, bally), axis=1)
        
        ball_idx= group.index[group['pred_class'] == 1]
        group.loc[ball_idx,'distance'] =  np.inf
        
        mask = group['pred_team'] > 1
        ppl_idx = group[mask].index
        group.loc[ppl_idx,'distance'] =  np.inf
        
        min_index = group['distance'].idxmin()
        group.loc[min_index, 'bp'] = 1
        group.loc[ball_idx,'distance'] = 0

        transformed_df.update(group)
        #transformed_df.loc[transformed_df["image_name"]==name]= group

        

    return transformed_df

bp_df = Ball_Possession(transformed_df)

#min distance between ball and player

In [19]:
bp_df

,pred_class,pred_x,pred_y,pred_w,pred_h,image_name,OID,pred_team,gnd_x,gnd_y,bp,distance
0,0.0,0.902366,0.795507,0.039593,0.097100,0.0,1.0,0.0,77.0,55.0,1.0,2.236068
1,0.0,0.640834,0.530382,0.016768,0.078459,0.0,2.0,1.0,73.0,38.0,0.0,16.124515
2,0.0,0.858254,0.601951,0.019514,0.098764,0.0,3.0,0.0,79.0,45.0,0.0,9.848858
3,0.0,0.237743,0.736137,0.020812,0.095080,0.0,4.0,0.0,57.0,50.0,0.0,18.439089
4,0.0,0.390634,0.391285,0.017607,0.066016,0.0,5.0,0.0,65.0,24.0,0.0,31.622777
...,...,...,...,...,...,...,...,...,...,...,...,...
14413,0.0,0.783640,0.308762,0.042718,0.117798,724.0,11.0,1.0,34.0,0.0,1.0,1.414214
14414,0.0,0.287292,0.367956,0.025289,0.114402,724.0,12.0,0.0,23.0,9.0,0.0,12.806248
14415,0.0,0.299241,0.446686,0.044959,0.116431,724.0,13.0,0.0,25.0,14.0,0.0,15.264338
14416,0.0,0.401181,0.190410,0.028988,0.099469,724.0,14.0,2.0,23.0,0.0,0.0,inf


## Pass Receiver Prediction using GNN

We utilize our best GNN (GATv2) for pass receiver prediction. Using the dataframe, we construct player graph(s) and feed into the model for prediction. The output is then converted back to a dataframe.

In [20]:
#Reciever Identification
from gnn.gnn_models import *
from  gnn.utils import *
import gnn.gnn_datagen as ggen

import importlib
importlib.reload(ggen)

bp_df['br']=0
file_path = "./processed/data.pt"
if os.path.exists(file_path):
    os.remove(file_path)


graph_list=ggen.csv2data(csv_df="df2",inp_df=bp_df,label_smoothing=0.0,graphfunc=2,undirected=False,negEdge=True)
dataset=MyDataset("./",graph_list)

val_loader = DataLoader(dataset, batch_size=1)



model = GAT2(hidden_channels= [32,32,32],heads=5,dropout=0)
PATH = './gnn/model_weights/best_model_GATv2_pre_ntrain_5.pt'



# model = GCN(hidden_channels= [16,16,16])
# PATH = './gnn/model_weights/best_model_GCNConv_pre_train.pt'

state = torch.load(PATH)
model.load_state_dict(state['state_dict'])



#Convert ID+x+y+team+bp+br 

Processed 537 objects and 0 skipped


Processing...
Done!


<All keys matched successfully>

In [21]:
for val_data in val_loader:
    output = model(val_data)
    pred_br= torch.argmax(output)
    bp_df.loc[(bp_df['image_name'] == float(val_data.name[0])) & (bp_df['OID'] == val_data.oid[0][pred_br]), 'br'] = 1 



In [22]:
bp_df

,pred_class,pred_x,pred_y,pred_w,pred_h,image_name,OID,pred_team,gnd_x,gnd_y,bp,distance,br
0,0.0,0.902366,0.795507,0.039593,0.097100,0.0,1.0,0.0,77.0,55.0,1.0,2.236068,0
1,0.0,0.640834,0.530382,0.016768,0.078459,0.0,2.0,1.0,73.0,38.0,0.0,16.124515,0
2,0.0,0.858254,0.601951,0.019514,0.098764,0.0,3.0,0.0,79.0,45.0,0.0,9.848858,1
3,0.0,0.237743,0.736137,0.020812,0.095080,0.0,4.0,0.0,57.0,50.0,0.0,18.439089,0
4,0.0,0.390634,0.391285,0.017607,0.066016,0.0,5.0,0.0,65.0,24.0,0.0,31.622777,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14413,0.0,0.783640,0.308762,0.042718,0.117798,724.0,11.0,1.0,34.0,0.0,1.0,1.414214,0
14414,0.0,0.287292,0.367956,0.025289,0.114402,724.0,12.0,0.0,23.0,9.0,0.0,12.806248,0
14415,0.0,0.299241,0.446686,0.044959,0.116431,724.0,13.0,0.0,25.0,14.0,0.0,15.264338,0
14416,0.0,0.401181,0.190410,0.028988,0.099469,724.0,14.0,2.0,23.0,0.0,0.0,inf,0


In the final predictions, the 'br' column represents whether the player has been predicted to receive the pass. <br>

## Export Video

We finally visualize the predictions on each frame of the video and save the output (.mp4 format).

In [23]:
# Write final predictions from dataframe to video for visualization
import imutils

def draw_boxes_on_frame(frame, boxes):
    for box in boxes:
        color = (0, 0, 255) if box['bp'] == 1 else (0, 255, 0)
        x, y, w, h = box['pred_x'], box['pred_y'], box['pred_w'], box['pred_h']
        x1, y1 = int((x - w / 2) * frame.shape[1]), int((y - h / 2) * frame.shape[0])
        x2, y2 = int((x + w / 2) * frame.shape[1]), int((y + h / 2) * frame.shape[0])
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        attributes = []
        if box['bp'] == 1:
            attributes.append('possession')
        if box['br'] == 1:
            attributes.append('receiver')
        attributes_str = ', '.join(attributes)
        cv2.putText(frame, attributes_str, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
    bp_boxes = [box for box in boxes if box['bp'] == 1]
    br_boxes = [box for box in boxes if box['br'] == 1]
    if bp_boxes and br_boxes:
        bp_box = bp_boxes[0]
        br_box = br_boxes[0]
        bp_x, bp_y, bp_w, bp_h = bp_box['pred_x'], bp_box['pred_y'], bp_box['pred_w'], bp_box['pred_h']
        br_x, br_y, br_w, br_h = br_box['pred_x'], br_box['pred_y'], br_box['pred_w'], br_box['pred_h']
        bp_center_x, bp_center_y = int(bp_x * frame.shape[1]), int(bp_y * frame.shape[0])
        br_center_x, br_center_y = int(br_x * frame.shape[1]), int(br_y * frame.shape[0])
        arrow_color = (255, 255, 255)
        cv2.arrowedLine(frame, (bp_center_x, bp_center_y), (br_center_x, br_center_y), arrow_color, 3)
        
    return frame

video_path = 'downloads/sample1.mp4'
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'sample1_annotated.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_boxes = bp_df[bp_df['image_name'] == frame_number]
    bp_boxes = frame_boxes[frame_boxes['bp'] == 1]
    br_boxes = frame_boxes[frame_boxes['br'] == 1]
    
    combined_boxes = pd.concat([bp_boxes, br_boxes], ignore_index=True, sort=False)
    
    if not combined_boxes.empty:
        frame = draw_boxes_on_frame(frame, combined_boxes.to_dict('records'))

    out.write(frame)

cap.release()
out.release()